## SimSiamの写経  
https://www.guruguru.science/competitions/17/discussions/a39d588e-aff2-4728-8323-b07f15563552/

In [1]:
import math
import torch
import torch.nn as nn
import torchvision
import numpy as np
import lightly
import os

In [2]:
num_workers = 4
batch_size = 128
seed = 1993
epochs = 50
input_size = 224

# dimension of the embeddings
num_ftrs = 512
# dimension of the output of the prediction and projection heads
out_dim = proj_hidden_dim = 512
# the prediction head uses a bottleneck architecture
#pred_hidden_dim = 128
# use 2 layers in the projection head
num_mlp_layers = 2

In [3]:
torch.manual_seed(0)
np.random.seed(0)

input_dir = '/src/atmacup/atmacup11/data/inputs/'
path_to_data = os.path.join(input_dir, 'photos')
model_dir = '/src/atmacup/atmacup11/data/model/'

## DataLoader

In [4]:
# define the augmentations for self-supervised learning
collate_fn = lightly.data.ImageCollateFunction(
    input_size=input_size,
    # require invariance to flips and rotations
    hf_prob=0.5,
    vf_prob=0.5,
    rr_prob=0.5,
    # satellite images are all taken from the same height
    # so we use only slight random cropping
    min_scale=0.5,
    # use a weak color jitter for invariance w.r.t small color changes
    cj_prob=0.2,
    cj_bright=0.1,
    cj_contrast=0.1,
    cj_hue=0.1,
    cj_sat=0.1,
)

# create a lightly dataset for training, since the augmentations are handled
# by the collate function, there is no need to apply additional ones here
dataset_train_simsiam = lightly.data.LightlyDataset(
    input_dir=path_to_data
)

# create a dataloader for training
dataloader_train_simsiam = torch.utils.data.DataLoader(
    dataset_train_simsiam,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers
)

# create a torchvision transformation for embedding the dataset after training
# here, we resize the images to match the input size during training and apply
# a normalization of the color channel based on statistics from imagenet
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((input_size, input_size)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=lightly.data.collate.imagenet_normalize['mean'],
        std=lightly.data.collate.imagenet_normalize['std'],
    )
])



# create a lightly dataset for embedding
dataset_test = lightly.data.LightlyDataset(
    input_dir=path_to_data,
    transform=test_transforms
)



# create a dataloader for embedding
dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

## Model

In [5]:
# we use a pretrained resnet for this tutorial to speed
# up training time but you can also train one from scratch
# Do not use pretrained Model
resnet = torchvision.models.resnet18(pretrained=False)
backbone = nn.Sequential(*list(resnet.children())[:-1])

# create the SimSiam model using the backbone from above
model = lightly.models.SimSiam(
    backbone,
    num_ftrs=num_ftrs,
#     proj_hidden_dim=pred_hidden_dim,
#     pred_hidden_dim=pred_hidden_dim,
#     out_dim=out_dim,
    num_mlp_layers=num_mlp_layers
)

## Loss / Optimizer

In [6]:
# SimSiam uses a symmetric negative cosine similarity loss
criterion = lightly.loss.SymNegCosineSimilarityLoss()

# scale the learning rate
lr = 0.05 * batch_size / 256
# use SGD with momentum and weight decay
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=lr,
    momentum=0.9,
    weight_decay=5e-4
)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

avg_loss = 0.
avg_output_std = 0.
for e in range(epochs):

    for (x0, x1), _, _ in dataloader_train_simsiam:

        # move images to the gpu
        x0 = x0.to(device)
        x1 = x1.to(device)

        # run the model on both transforms of the images
        # the output of the simsiam model is a y containing the predictions
        # and projections for each input x
        y0, y1 = model(x0, x1)

        # backpropagation
        loss = criterion(y0, y1)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        # calculate the per-dimension standard deviation of the outputs
        # we can use this later to check whether the embeddings are collapsing
        output, _ = y0
        output = output.detach()
        output = torch.nn.functional.normalize(output, dim=1)

        output_std = torch.std(output, 0)
        output_std = output_std.mean()

        # use moving averages to track the loss and standard deviation
        w = 0.9
        avg_loss = w * avg_loss + (1 - w) * loss.item()
        avg_output_std = w * avg_output_std + (1 - w) * output_std.item()

    # the level of collapse is large if the standard deviation of the l2
    # normalized output is much smaller than 1 / sqrt(dim)
    collapse_level = max(0., 1 - math.sqrt(out_dim) * avg_output_std)
    # print intermediate results
    print(f'[Epoch {e:3d}] '
        f'Loss = {avg_loss:.2f} | '
        f'Collapse Level: {collapse_level:.2f} / 1.00')

[Epoch   0] Loss = -0.73 | Collapse Level: 0.55 / 1.00
[Epoch   1] Loss = -0.84 | Collapse Level: 0.56 / 1.00
[Epoch   2] Loss = -0.85 | Collapse Level: 0.55 / 1.00
[Epoch   3] Loss = -0.87 | Collapse Level: 0.55 / 1.00
[Epoch   4] Loss = -0.88 | Collapse Level: 0.55 / 1.00
[Epoch   5] Loss = -0.90 | Collapse Level: 0.56 / 1.00
[Epoch   6] Loss = -0.90 | Collapse Level: 0.56 / 1.00
[Epoch   7] Loss = -0.91 | Collapse Level: 0.56 / 1.00
[Epoch   8] Loss = -0.90 | Collapse Level: 0.56 / 1.00
[Epoch   9] Loss = -0.90 | Collapse Level: 0.56 / 1.00
[Epoch  10] Loss = -0.91 | Collapse Level: 0.56 / 1.00
[Epoch  11] Loss = -0.90 | Collapse Level: 0.56 / 1.00
[Epoch  12] Loss = -0.91 | Collapse Level: 0.56 / 1.00
[Epoch  13] Loss = -0.91 | Collapse Level: 0.56 / 1.00
[Epoch  14] Loss = -0.91 | Collapse Level: 0.56 / 1.00
[Epoch  15] Loss = -0.91 | Collapse Level: 0.56 / 1.00
[Epoch  16] Loss = -0.91 | Collapse Level: 0.56 / 1.00
[Epoch  17] Loss = -0.91 | Collapse Level: 0.56 / 1.00
[Epoch  18

In [ ]:
from torchsummary import summary
summary(model, (3,224,224))

In [ ]:
torch.save(model.state_dict(), os.path.join(model_dir,'simsiam_res18.pth'))

In [ ]:
model.load_state_dict(torch.load(os.path.join(model_dir,'simsiam_res18.pth')))